In [ ]:
columns_with_missing_values = get_columns_with_missing_values(arg_di_df)
columns_with_missing_values

['sm_pop_refg_or',
 'se_ter_enrr',
 'se_pre_enrr',
 'se_prm_enrr',
 'se_sec_enrr',
 'sp_dyn_le00_in',
 'sh_dth_imrt']

In [ ]:
print("Testing accuracy =", accuracy_score(ytest, ypred))